<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/LLMs_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sacrebleu bert-score
!pip install sacrebleu bert-score pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 28.9 MB/s eta 0:00:00


In [ ]:
from typing import List
import sacrebleu
from sacrebleu.metrics import BLEU
from bert_score import score as bert_score

# --- Your example sentences ---
references: List[str] = [
    "วันนี้อากาศดีและสดชื่น",
]
candidates: List[str] = [
    "อากาศวันนี้สดชื่นและน่ารื่นรมย์",
]

# =============== BLEU helpers ===============
def bleu_sacrebleu(hyps: List[str], refs: List[str], tokenizer: str = "intl") -> float:
    """
    Use sacreBLEU with 'intl' for multilingual/Thai.
    Returns BLEU score on 0..100 scale.
    """
    bleu = BLEU(tokenize=tokenizer)  # create BLEU scorer
    return bleu.corpus_score(hyps, [refs]).score

def bleu_with_thai_tokenization(hyps: List[str], refs: List[str]) -> float:
    """
    Thai word segmentation to add spaces before BLEU.
    """
    try:
        from pythainlp.tokenize import word_tokenize
    except Exception as e:
        raise RuntimeError("Please `pip install pythainlp` to use Thai tokenization.") from e

    seg = lambda s: " ".join(word_tokenize(s, engine="newmm"))
    hyps_seg = [seg(x) for x in hyps]
    refs_seg = [seg(x) for x in refs]

    bleu = BLEU(tokenize="13a")  # English-style now works after segmentation
    return bleu.corpus_score(hyps_seg, [refs_seg]).score

# =============== BERTScore helper ===============
def bertscore(hyps: List[str], refs: List[str],
              model_type: str = "xlm-roberta-large") -> dict:
    """
    Use a multilingual model for Thai (XLM-R large is a strong default).
    Returns mean Precision/Recall/F1 in 0..1 range.
    """
    P, R, F = bert_score(hyps, refs, model_type=model_type, verbose=False)
    return {
        "P": P.mean().item(),
        "R": R.mean().item(),
        "F1": F.mean().item(),
    }

# =============== Evaluate ===============
print("Pairs:", len(references))

# BLEU (multilingual, no extra tokenization)
bleu_intl = bleu_sacrebleu(candidates, references, tokenizer="intl")
print(f"BLEU (intl tokenizer): {bleu_intl:.2f}")

# BLEU with Thai word segmentation
try:
    bleu_thseg = bleu_with_thai_tokenization(candidates, references)
    print(f"BLEU (with Thai tokenization): {bleu_thseg:.2f}")
except RuntimeError as e:
    print("(Skip tokenized BLEU) ->", e)

# BERTScore (semantic)
b = bertscore(candidates, references, model_type="xlm-roberta-large")
print(f"BERTScore-P: {b['P']:.4f}  BERTScore-R: {b['R']:.4f}  BERTScore-F1: {b['F1']:.4f}")


Pairs: 1
BLEU (intl tokenizer): 0.00
BLEU (with Thai tokenization): 15.11
BERTScore-P: 0.9522  BERTScore-R: 0.9591  BERTScore-F1: 0.9556


##################


In [ ]:
# ---- Install (Colab) ----
!pip -q install sacrebleu bert-score pythainlp pandas openpyxl tqdm

import re
import pandas as pd
from tqdm import tqdm
from typing import List, Tuple
from sacrebleu.metrics import BLEU
from bert_score import score as bert_score

# ====== CONFIG ======
XLSX_PATH = "/MMT4NL_result.xlsx"   # เปลี่ยนเป็นพาธไฟล์ของคุณ (หรืออัปโหลดทาง Colab)
SAVE_PATH = "/content/MMT4NL_result_scored.xlsx"


In [ ]:
# ระบุชื่อคอลัมน์ ถ้าทราบแน่นอน (แนะนำ)
REF_COL = None        # เช่น "reference" หรือ "gold"
HYP_COL = None        # เช่น "candidate" หรือ "prediction"

# ภาษาไทย: ใช้ตัดคำและ tokenizer ที่เหมาะสม
USE_THAI_WORD_SEG = True            # True = ตัดคำไทยก่อนคำนวณ BLEU
BERT_MODEL = "xlm-roberta-large"    # multilingual เหมาะกับไทย
BLEU_TOKENIZER_NOSEG = "intl"       # ถ้าไม่ตัดคำ ใช้ intl
BLEU_TOKENIZER_SEG   = "13a"        # ถ้าตัดคำแล้ว ใช้ 13a



In [ ]:
# ====== Helpers ======
def find_cols(df: pd.DataFrame) -> Tuple[str, str]:
    """เดาชื่อคอลัมน์อัตโนมัติ ถ้าไม่ระบุไว้ใน CONFIG"""
    cols = [c.lower() for c in df.columns]
    ref_candidates = ["reference", "ref", "target", "gold", "ground", "groundtruth", "gt", "answer"]
    hyp_candidates = ["candidate", "hypothesis", "hyp", "prediction", "pred", "output", "system"]
    ref = next((c for c in df.columns if c.lower() in ref_candidates), None)
    hyp = next((c for c in df.columns if c.lower() in hyp_candidates), None)
    if ref is None or hyp is None:
        # ลองเดาจากคำหลัก
        ref = ref or next((c for c in df.columns if re.search(r"ref|gold|answer|target", c, re.I)), None)
        hyp = hyp or next((c for c in df.columns if re.search(r"hyp|cand|pred|out|system", c, re.I)), None)
    if ref is None or hyp is None:
        raise ValueError(
            f"ไม่พบคอลัมน์อ้างอิง/ผลโมเดลในชีตครับ โปรดตั้ง REF_COL/HYP_COL ให้ชัด "
            f"หรือเปลี่ยนชื่อหัวคอลัมน์ให้มีคำว่า reference/candidate (ปัจจุบันมี: {list(df.columns)})"
        )
    return hyp, ref

def thai_segment(texts: List[str]) -> List[str]:
    from pythainlp.tokenize import word_tokenize
    seg = lambda s: " ".join(word_tokenize(str(s), engine="newmm"))
    return [seg(t if isinstance(t, str) else "") for t in texts]

def compute_bleu_per_sentence(hyps: List[str], refs: List[str], use_seg: bool) -> List[float]:
    """
    คำนวณ BLEU แบบประโยคต่อประโยค (sentence-level) พร้อม smoothing
    - ถ้า use_seg=True จะตัดคำไทยก่อน แล้วใช้ tokenizer='13a'
    - ถ้า use_seg=False จะไม่ตัดคำ ใช้ tokenizer='intl'
    """
    if use_seg:
        hyps_proc = thai_segment(hyps)
        refs_proc = thai_segment(refs)
        bleu = BLEU(tokenize=BLEU_TOKENIZER_SEG, smooth_method="exp")
    else:
        hyps_proc, refs_proc = hyps, refs
        bleu = BLEU(tokenize=BLEU_TOKENIZER_NOSEG, smooth_method="exp")

    scores = []
    for h, r in zip(hyps_proc, refs_proc):
        # sentence_score ควรใส่ refs เป็น list[list[str]] สำหรับหลาย reference
        s = bleu.sentence_score(h, [r])
        scores.append(s.score)  # 0..100
    return scores

def compute_bertscore_per_sentence(hyps: List[str], refs: List[str], model_type: str) -> pd.DataFrame:
    """
    คำนวณ BERTScore แบบ batch ทั้งชุด แล้วแตกค่าต่อบรรทัด
    คืน DataFrame มีคอลัมน์: bert_P, bert_R, bert_F1 (0..1)
    """
    P, R, F = bert_score(hyps, refs, model_type=model_type, verbose=True)
    return pd.DataFrame({
        "BERT_P":  P.tolist(),
        "BERT_R":  R.tolist(),
        "BERT_F1": F.tolist(),
    })



In [ ]:
# ========= Load =========
df = pd.read_excel(XLSX_PATH)
print("Columns:")
for i, c in enumerate(df.columns):
    print(f"{i:>2}: {c}")

# ======= CHOOSE COLUMNS HERE =======
# เลือก "reference" (มาตรฐานทอง): ใส่ชื่อคอลัมน์ หรือใส่เลข index ตามที่พิมพ์ด้านบน
# ตัวอย่างที่เห็นในสกรีนช็อต: อาจใช้คอลัมน์ข้อความต้นฉบับภาษาไทยเป็น reference
REF_COL = "translate eng to thai (gpt 4o)"      # <-- แก้ให้ตรงของคุณ
# หรือใช้ index แทน เช่น REF_COL = df.columns[?]

# เลือกหลายคอลัมน์ของ “ผลลัพธ์โมเดล” เพื่อเปรียบเทียบพร้อมกัน (คำนวณให้ทุกคอลัมน์)
HYP_COLS = [
    "translate eng to thai (typhoon)",          # <-- แก้/เพิ่มได้ตามต้องการ
    # "translate thai to eng(typhoon)",         # ตัวอย่างเพิ่ม
]
# ถ้าอยากเลือกด้วยดัชนี: HYP_COLS = [df.columns[3], df.columns[6]]

Columns:
 0: Unnamed: 0
 1: Sentiment
 2: translate thai to eng(typhoon)
 3: Prompt: https://chatgpt.com/share/67cf709b-eb64-8004-b4b4-ab0952f5f375
 4: translate eng to thai (typhoon)
 5: BLEU
 6: BERT score
 7: translate eng to thai (gpt 4o)
 8: Prompt: https://chatgpt.com/share/67cb7fa2-5e88-8004-bb5a-8f79db7a46f7
 9: Prompt: https://chatgpt.com/share/67cf8dbe-0358-8004-9064-06a64d22f2f6
10: update compare result (Neutral is accepted)
11: Prompt: https://chatgpt.com/share/67cf8f8d-e5a8-8004-978f-4f7bede836b5
12: Prompt: https://chatgpt.com/share/67cf8f79-37b0-8004-8c3f-fd0c4906020a
13: Unnamed: 13
14: Unnamed: 14
15: Unnamed: 15


In [ ]:
# ======= Validate =======
if REF_COL not in df.columns:
    raise ValueError(f"REF_COL '{REF_COL}' not found. ใช้ชื่อคอลัมน์ให้ตรงหรือเปลี่ยนเป็นดัชนีเช่น df.columns[3].")
for col in HYP_COLS:
    if col not in df.columns:
        raise ValueError(f"HYP_COL '{col}' not found. ตรวจสอบชื่อคอลัมน์: {list(df.columns)}")

ref_list = df[REF_COL].astype(str).fillna("").tolist()

# ลบผลเก่า (ถ้ามี) เพื่อไม่ให้ซ้ำ
for c in list(df.columns):
    if re.search(r"^BLEU_sent|^BERT_", c):
        df.drop(columns=[c], inplace=True, errors="ignore")


In [ ]:
# ======= Compute for each hypothesis column =======
for hyp_col in HYP_COLS:
    hyps = df[hyp_col].astype(str).fillna("").tolist()
    print(f"\nScoring against REF='{REF_COL}'  HYP='{hyp_col}'")

    df[f"BLEU_sent :: {hyp_col}"] = bleu_sentence_list(hyps, ref_list, use_seg=USE_THAI_WORD_SEG)

    bdf = bertscore_df(hyps, ref_list, model_type=BERT_MODEL)
    df[f"BERT_P :: {hyp_col}"]  = bdf["BERT_P"]
    df[f"BERT_R :: {hyp_col}"]  = bdf["BERT_R"]
    df[f"BERT_F1 :: {hyp_col}"] = bdf["BERT_F1"]



Scoring against REF='translate eng to thai (gpt 4o)'  HYP='translate eng to thai (typhoon)'


In [ ]:
df.to_excel(SAVE_PATH, index=False)
print(f"\n✅ Saved to: {SAVE_PATH}")


✅ Saved to: /content/MMT4NL_result_scored.xlsx


In [ ]:
from google.colab import files
files.download("/content/MMT4NL_result_scored.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,Unnamed: 0,Sentiment,translate thai to eng(typhoon),Prompt: https://chatgpt.com/share/67cf709b-eb64-8004-b4b4-ab0952f5f375,translate eng to thai (typhoon),BLEU,BERT score,translate eng to thai (gpt 4o),Prompt: https://chatgpt.com/share/67cb7fa2-5e88-8004-bb5a-8f79db7a46f7,Prompt: https://chatgpt.com/share/67cf8dbe-0358-8004-9064-06a64d22f2f6,update compare result (Neutral is accepted),Prompt: https://chatgpt.com/share/67cf8f8d-e5a8-8004-978f-4f7bede836b5,Prompt: https://chatgpt.com/share/67cf8f79-37b0-8004-8c3f-fd0c4906020a,Unnamed: 13,Unnamed: 14,Unnamed: 15,BLEU_sent :: translate eng to thai (typhoon),BERT_P :: translate eng to thai (typhoon),BERT_R :: translate eng to thai (typhoon),BERT_F1 :: translate eng to thai (typhoon)
0,sentimentID,NaN,NaN,taxonomy,NaN,NaN,NaN,NaN,ner,negation,negation (updated),vocab,fairness,robustness,temporal,temporal (updated),0.000000,1.000000,1.000000,1.000000
1,1,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,I am very happy to meet new friends.,I am very delighted to meet new friends.,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,NaN,NaN,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,Lily is very happy to meet new friends.,I am very not unhappy to meet new friends.,NaN,I am very truly happy to meet new friends.,She is very happy to meet new friends.,NaN,NaN,NaN,100.000000,1.000000,1.000000,1.000000
2,2,วันนี้อากาศดีและสดชื่น,The weather today is good and refreshing.,The weather today is pleasant and refreshing.,อากาศวันนี้สดชื่นและน่ารื่นรมย์,NaN,NaN,วันนี้อากาศดีและสดชื่น,The weather today is good and refreshing.,The weather today is not bad and refreshing.,NaN,The weather today is good and wonderfully refr...,The weather today is good and refreshing for a...,NaN,NaN,NaN,15.106877,0.952192,0.959062,0.955615
3,3,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,She feels sad that she can't go to the event.,She feels upset that she can’t go to the event.,เธอเสียใจที่ไม่สามารถไปร่วมงานได้,NaN,NaN,เธอรู้สึกเสียใจที่ไม่สามารถไปงานได้,Emma feels sad that she can’t go to the event.,She feels not happy that she can’t go to the e...,NaN,She feels truly sad that she can’t go to the e...,The Thai girl feels sad that she can’t go to t...,NaN,NaN,NaN,49.626448,0.960317,0.958576,0.959446
4,4,อาหารมื้อนี้อร่อยสุดๆ,This meal is extremely delicious.,This meal is extremely scrumptious.,อาหารมื้อนี้อร่อยอย่างยิ่ง,NaN,NaN,อาหารมื้อนี้อร่อยมาก,This meal is extremely delicious.,This meal is not bland.,NaN,This meal is extremely absolutely delicious.,This Japanese meal is extremely delicious.,NaN,NaN,NaN,66.874030,0.981483,0.980838,0.981160
5,5,ฉันไม่พอใจกับบริการที่ได้รับ,I am not satisfied with the service I received.,I am not pleased with the service I received.,ฉันไม่พอใจกับบริการที่ได้รับ,NaN,NaN,ฉันไม่พอใจกับบริการที่ได้รับ,Ben is not satisfied with the service he recei...,I am unsatisfied with the service I received.,NaN,I am not fully satisfied with the service I re...,"I, as a Vietnamese customer, am not satisfied ...",NaN,NaN,NaN,100.000000,1.000000,1.000000,1.000000
6,NaN,หนังเรื่องนี้น่าเบื่อและยืดเยื้อ,This movie is boring and drawn out.,This movie is boring and lengthy.,หนังเรื่องนี้ไม่สนุกและยาวเกินไป,NaN,NaN,หนังเรื่องนี้น่าเบื่อและยืดเยื้อ,This movie is boring and drawn out.,This movie is not entertaining and drawn out.,NaN,This movie is boring and overly drawn out.,This Chinese movie is boring and drawn out.,NaN,NaN,NaN,22.089591,0.921924,0.897250,0.909420
7,NaN,เขาตื่นเต้นกับโอกาสครั้งนี้,He is excited about this opportunity.,He is thrilled about this opportunity.,เขาตื่นเต้นกับโอกาสนี้มาก,NaN,NaN,เขารู้สึกตื่นเต้นกับโอกาสนี้,Leo is excited about this opportunity.,He is not uninterested about this opportunity.,NaN,He is genuinely excited about this opportunity.,The Indian boy is excited about this opportunity.,NaN,NaN,NaN,53.728497,0.969957,0.983332,0.976599
8,NaN,เขาทำงานหนักและได้ผลลัพธ์ที่ดี,He worked hard and got good results.,He worked diligently and got good results.,เขาทำงานอย่างขยันขันแข็งและได้ผลลัพธ์ที่ดี,NaN,NaN,เขาทำงานหนักและได้ผ

In [1]:
#BlUE
#print double check row 18